In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn, optim
%matplotlib inline
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [7]:
#Define plotting function

def get_hist(model, data_set):
    activations = model.activation(data_set.x)
    for i, activation in enumerate(activations):
        plt.hist(activation.numpy(), 4, density = True)
        plt.title("Activation layer" + str(i + 1))
        plt.xlabel("Activation")
        plt.ylabel("Prediction")
        plt.legend()
        plt.show()

In [8]:
def PlotStuff(X, Y, model = None, leg = False):
    
    plt.plot(X[Y==0].numpy(), Y[Y==0].numpy(), 'or', label = 'training points y = 0')
    plt.plot(X[Y==1].numpy(), Y[Y==1].numpy(), 'ob', label = 'training points y = 1')
    
    if model != None:
        plt.plot(X.numpy(), model(X).detach().numpy(), label = 'Neural network')
    
    plt.legend()
    plt.show()

In [9]:
#Define the class to get our data
#[r:c]
class data(Dataset):
    def __init__(self):
        self.x = torch.linspace(-20, 20, 100).view(-1, 1)
        self.y = torch.zeros(self.x.shape[0])
        self.y[(self.x[:,0] > -10) & (self.x[:,0] < -5)] = 1
        self.y[(self.x[:,0] > 5) & (slef.x[:0] < 10)] = 1
        self.y = self.y.view(-1, 1) ##??
        self.len = self.x.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.len
        

In [10]:
#Define the class for creating our model

class Net(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)
        
    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        return x

In [ ]:
def train(data_set, model, criterion, train_loader, optimizer, epochs = 5, plot_number = 10):
    cost = []
    
    for epoch in range(epochs):
        total = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            
            yhat = model(x)
            loss = criterion(yhat, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total+=loss.item()
            
        if epoch%plot_number ==0